In [1]:
import os
import sys
import numpy as np
from tqdm import tqdm
import time
import pickle

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

In [3]:
# path 추가
sys.path.append('/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/bertsum')

In [4]:
torch.__version__

'1.1.0'

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

## Load dataset

train_div, train_org<br>
val_div, val_org<br>
test_div, test_org<br>

In [6]:
window_size = 3

In [7]:
data_path = f'/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/subtext/dataset/nn_dataset_w{window_size}.pkl'

with open(data_path, 'rb') as rr:
    dataset = pickle.load(rr)

dataset_fin = [data.permute(0, 2, 1) for data in dataset]

train_x = torch.cat((dataset_fin[0], dataset_fin[1]), dim=0).to(device)
train_y = torch.cat((torch.ones(len(dataset_fin[0])), torch.zeros(len(dataset_fin[1]))), dim=0).to(device)

val_x = torch.cat((dataset_fin[2], dataset_fin[3]), dim=0).to(device)
val_y = torch.cat((torch.ones(len(dataset_fin[2])), torch.zeros(len(dataset_fin[3]))), dim=0).to(device)

test_x = torch.cat((dataset_fin[4], dataset_fin[5]), dim=0).to(device)
test_y = torch.cat((torch.ones(len(dataset_fin[4])), torch.zeros(len(dataset_fin[5]))), dim=0).to(device)

# batch iterator
batch_size = 256

trainset = data_utils.TensorDataset(train_x, train_y)
train_loader = data_utils.DataLoader(trainset, batch_size = batch_size, shuffle = True)

valset = data_utils.TensorDataset(val_x, val_y)
val_loader = data_utils.DataLoader(valset, batch_size = len(valset), shuffle = True)

testset = data_utils.TensorDataset(test_x, test_y)
test_loader = data_utils.DataLoader(testset, batch_size = len(testset), shuffle = True)

# Model

In [15]:
class ChunkClassifier(nn.Module):
    
    def __init__(self, x_features):
        super(ChunkClassifier, self).__init__()
        
        # block1
        self.block1 = nn.Sequential(
            nn.Conv1d(in_channels=768, out_channels=128, kernel_size=4),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(128*3, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            nn.Linear(16, 1)
        )
        
    def forward(self, x):
        
        # forward block 1
        out = self.block1(x)
        batch_size = out.size()[0]
        
        out = out.view(batch_size, -1)
        out = self.block2(out)
        
        return out.view(-1)

In [16]:
chunk_model = ChunkClassifier(x_features = 768)

chunk_model.to(device)

ChunkClassifier(
  (block1): Sequential(
    (0): Conv1d(768, 128, kernel_size=(4,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Linear(in_features=384, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [17]:
crit = nn.BCEWithLogitsLoss()
learning_rate = 1e-3

optimizer = optim.Adam(chunk_model.parameters(), lr = learning_rate)

decayRate = 0.998
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = decayRate)

num_batches = len(train_loader)

In [18]:
train_loss_list = []
val_loss_list = []
train_corrects_sum = 0.0
val_corrects_sum = 0.0

train_tot_num = train_loader.dataset.__len__()
val_tot_num = val_loader.dataset.__len__()

epochs = 10
for epoch in range(epochs):
    
    train_loss_sum = 0.0
    
    for i, train_block in enumerate(train_loader):
        
        train_X, train_Y = train_block[0], train_block[1]
        optimizer.zero_grad()
        
        # loss 계산
        train_pred = chunk_model(train_X)
        train_loss = crit(train_pred, train_Y)
        train_loss.backward()
        
        optimizer.step()
        
        # Prediction Accuracy
        train_pred_label = (train_pred > 0.0).float()
        train_corrects = (train_pred_label == train_Y).sum()
        train_corrects_sum += train_corrects.item()
        
        train_loss_sum += train_loss
        
    with torch.no_grad():
        val_loss_sum = 0.0
        for j, val_block in enumerate(val_loader):
            val_X, val_Y = val_block[0], val_block[1]

            val_pred = chunk_model(val_X)
            val_loss = crit(val_pred, val_Y)
            val_loss_sum += val_loss

            val_pred_label = (val_pred > 0.0).float()
            val_corrects = (val_pred_label == val_Y).sum()
            val_corrects_sum += val_corrects.item()

    train_loss_sum = 0.0
    
    print(f"Epoch {epoch+1} done.")
    print(f"{train_corrects_sum/train_tot_num:.4f}, {val_corrects_sum/val_tot_num:.4f}\n")
    
    train_corrects_sum = 0.0
    val_corrects_sum = 0.0
    
    # learning rate decaying
    lr_scheduler.step()

Epoch 1 done.
0.9515, 0.9755

Epoch 2 done.
0.9798, 0.9804

Epoch 3 done.
0.9848, 0.9819

Epoch 4 done.
0.9875, 0.9823

Epoch 5 done.
0.9905, 0.9815

Epoch 6 done.
0.9927, 0.9799

Epoch 7 done.
0.9955, 0.9815

Epoch 8 done.
0.9943, 0.9827

Epoch 9 done.
0.9967, 0.9828

Epoch 10 done.
0.9977, 0.9833



In [19]:
# test
chunk_model.eval()

test_tot_num = test_loader.dataset.__len__()
for i, test_block in enumerate(test_loader):
    test_X, test_Y = test_block[0], test_block[1]

    test_pred = chunk_model(test_X)
    test_pred_label = (test_pred > 0.0).float()
    test_corrects = (test_pred_label == test_Y).sum()
    
    print(f"Test Accuracy: {test_corrects.item()/test_tot_num:.4f}")

Test Accuracy: 0.9800


In [21]:
# save model
save_pth = f'/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/subtext/chunk_nn/ckpt/chunk_model_w{window_size}_ckpt.pt'
torch.save(chunk_model.state_dict(), save_pth)
print(f"Model Saved at: {save_pth}")

Model Saved at: /home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/subtext/chunk_nn/ckpt/chunk_model_w3_ckpt.pt


# Test with real data

In [22]:
# Load model
window_size = 3
model_pth = f'/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/subtext/chunk_nn/ckpt/chunk_model_w{window_size}_ckpt.pt'

chunk_model.load_state_dict(torch.load(model_pth))
chunk_model.eval()

ChunkClassifier(
  (block1): Sequential(
    (0): Conv1d(768, 128, kernel_size=(4,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Linear(in_features=384, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=1, bias=True)
  )
)

## Load BertSum

In [23]:
from models.data_loader import TextLoader, load_dataset
from bertsum import args, ExtTransformerEncoder, ExtSummarizer, WindowEmbedder

In [24]:
# Settings
device = "cpu" if args.visible_gpus == -1 else "cuda"
loader = TextLoader(args, device)

# model setting
ckpt_path = '/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/bertsum/checkpoint/model_step_24000.pt'
checkpoint = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model = ExtSummarizer(args, device, checkpoint)
model.eval()

using cached model
using cached model
using cached model
using cached model
using cached model


ExtSummarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(8004, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): LayerNorm(torch.Size([768]),

In [25]:
embedder = WindowEmbedder(model=model, text_loader=loader)

### (1) 기사 데이터

In [8]:
import json
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [9]:
data_path = '/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/bertsum/dataset'
news_df = load_jsonl(os.path.join(data_path, 'train.jsonl'))

Loaded 260697 records from /home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/bertsum/dataset/train.jsonl


In [10]:
# 전처리
# (1) 글자 개수가 너무 작은 경우 없애기 (30글자 이상)
# (2) 문장이 적은 경우 해당 기사 없애기 (10문장 이상)
news_clean = []
for news in news_df:
    news_article = news['article_original']
    if len(news_article) >= 10:
        article_clean = [sent for sent in news_article]
        news_clean.append(article_clean)

In [29]:
test_article="""
무주택자는 우대를 받아 여기에 10%를 더 빌릴 수 있는데 이런 우대를 더 많은 대상에게 확대 적용하겠다는 겁니다.
당정은 소득을 따져서 부채 규모를 제한하는 dsr 규제도 완화하는 방안을 앞으로 논의할 계획입니다.
국회 정무위 민주당 간사인 김병욱 의원은 1가구 1주택 종합부동산세 적용 기준을 현행 9억 원에서 12억 원으로 상향하는 법안을 발의했습니다.
민주당에서는 재보선 패배 이후 부동산 세제를 완화하자는 주장이 개별적인 차원이지만 이어지고 있습니다.
국민의 힘은 늦어지는 백신 수급에 문제를 제기하며 다음 달 한미정상회담에서 문재인 대통령이 코로나19 백신을 충분히 확보해야 한다고 지적했습니다.
국민의 힘 주호영 당 대표 권한대행은 미국 방문의 가장 중요한 의제는 백신 확보가 돼야 한다면서 좋은 코로나19 백신을 어떻게 많이 확보하느냐에 우리 외교력의 성적표가 달려있다고 강조했습니다.
정부가 올해 2분기에 도입하겠다고 밝혔던 모더나 백신은 상반기 확보가 어려울 전망입니다.
홍남기 국무총리 대행은 오늘 국회 대정부질문에서 문 대통령이 모두나 최고경영자와 화상통화를 확보했다는 모두나 백신은 어디 있느냐는 국민의 힘 김은혜 의원의 질문에 이같이 답했습니다.
""".split('\n')

test_article = [sent for sent in test_article if sent]

In [149]:
test_article="""
무주택자는 우대를 받아 여기에 10%를 더 빌릴 수 있는데 이런 우대를 더 많은 대상에게 확대 적용하겠다는 겁니다.
당정은 소득을 따져서 부채 규모를 제한하는 dsr 규제도 완화하는 방안을 앞으로 논의할 계획입니다.
국회 정무위 민주당 간사인 김병욱 의원은 1가구 1주택 종합부동산세 적용 기준을 현행 9억 원에서 12억 원으로 상향하는 법안을 발의했습니다.
민주당에서는 재보선 패배 이후 부동산 세제를 완화하자는 주장이 개별적인 차원이지만 이어지고 있습니다.
신고를 받은 경찰이 도착해 가까스로 상황이 일단락됩니다.
피해 직원은 본인을 도둑 취급했다는 것에 분노한 것은 이해할 수 있지만 그렇다고 폭력이 정당화될 수는 없다고 kbs에 말했습니다.
명백한 폭행 사건으로 보이지만 처벌이 가능할지는 미지수입니다 우리나라가 가입한 외교관계에 관한 비엔나 협약에 외교관의 가족은 주재국의 형사 재판을 받지 않는다고 규정돼 있어서입니다.
우리나라 법정에서 재판을 받으려면 벨기에 정부가 이런 면책 특권을 포기하는 방법밖에 없지만 그럴 가능성은 없어 보입니다.
""".split('\n')

test_article = [sent for sent in test_article if sent]

In [213]:
test_article="""
특허받은 콘덴서 자동세척 시스템 건조 때마다 1~3회 자동세척 청소할 필요가 없다는 등의 광고 문구가 등장합니다.
하지만 이런 기능에도 불구하고 적잖은 소비자들이 기계 내 먼지 사인과 악취 등을 호소했고 결국 lg전자는 리콜과 무상보증을 결정했습니다.
이불 털기나 소량 건조 등 상황에 따라 자동세척 시스템이 작동하지 않는 문제가 있었던 겁니다.
공정거래위원회는 이를 토대로 당시 lg전자의 광고가 거짓 과장 광고에 해당한다고 결론냈습니다.
광고를 본 소비자들은 자동 세척 시스템만 있으면 언제나 깨끗한 상태의 건조기를 쓸 수 있을 걸로 오인하게 된다는 겁니다.
lg전자 측은 깨끗하게와 같은 표현은 실증의 대상이 아니며 자동 세척이 안 되는 건 소량 빨래 같은 예외적인 상황이었다고 주장했지만 받아들여지지 않았습니다.
자동세척 시스템의 성능 효과와 관련된 상임으로 실정이 대상이 되고 특히 일 아주 건조 목적의 공익이 증가하는 것 등을 고려하였을 때 소량 건조가 예외적인 상황은 아니라고 판단하였습니다.
공정위는 콘덴서 자동세척 기능이 건조기의 핵심 선택 기준 가운데 하나로 광고되는 등 소비자의 선택에 큰 영향을 미쳤다며 lg전자의 시정 공표 명령과 함께 과징금 3억 9천만 원을 부과했습니다.
""".split('\n')

test_article = [sent for sent in test_article if sent]

In [34]:
test_article="""
광고를 본 소비자들은 자동 세척 시스템만 있으면 언제나 깨끗한 상태의 건조기를 쓸 수 있을 걸로 오인하게 된다는 겁니다.
lg전자 측은 깨끗하게와 같은 표현은 실증의 대상이 아니며 자동 세척이 안 되는 건 소량 빨래 같은 예외적인 상황이었다고 주장했지만 받아들여지지 않았습니다.
자동세척 시스템의 성능 효과와 관련된 상임으로 실정이 대상이 되고 특히 일 아주 건조 목적의 공익이 증가하는 것 등을 고려하였을 때 소량 건조가 예외적인 상황은 아니라고 판단하였습니다.
공정위는 콘덴서 자동세척 기능이 건조기의 핵심 선택 기준 가운데 하나로 광고되는 등 소비자의 선택에 큰 영향을 미쳤다며 lg전자의 시정 공표 명령과 함께 과징금 3억 9천만 원을 부과했습니다.
식용 옥수수 수입 시 적용되는 관세율을 올 12월 31일까지 0%까지 인하하는 내용을 담은 탈당 관세 규정 개정안이 오늘 국무회의를 통과했습니다.
이에 따라 연말까지 수입되는 식용 옥수수 총 128만 톤에 대한 관세율이 기존 3%에서 0%로 내려갑니다.
이번 조치는 국제 곡물 가격 상승에 선제적으로 대응하는 취지에서 이뤄졌습니다.
기획재정부는 식용 옥수수가 전분 전분당으로 가공돼 제과 제빵 제면 등 식품 원료로 사용되기 때문에 이번 조치가 식품 가격 안정에 기여할 것으로 예상된다고 밝혔습니다.
""".split('\n')

test_article = [sent for sent in test_article if sent]

### (2) 유튜브 스크립트

In [26]:
from utils.preprocess import doc_preprocess
import json

In [27]:
youtube_script_pth = '/home/sks/korea_univ/21_1/TA/project/video_dataset/youtube-summarization/data/label/KBS뉴스_NSBlDZeLeLw_27m_36s.txt'
with open(youtube_script_pth, 'rb') as rr:
    youtube_df = json.load(rr)

In [28]:
script = youtube_df['text']

In [29]:
script_fin = doc_preprocess(script)

In [30]:
script_list = [sent for sent in script_fin.split('\n') if len(sent.strip()) >= 20][:50]
#script_list = [sent for sent in script_fin.split('\n')][:50]

In [32]:
ws = 3

In [33]:
chunk_model.eval()

score_list = []
for i, sent in enumerate(tqdm(script_list)):
    if i + (ws*2) <= len(script_list):
        w_input = script_list[i:i+(ws*2)]
        
        # embedding
        emb = embedder.get_embeddings(w_input).transpose(1, 0).cuda()
        score = chunk_model(emb.unsqueeze(0))
        score_list.append(score.item())

100%|██████████| 50/50 [00:06<00:00,  7.19it/s]


In [34]:
script_list

['여러분 안녕하십니까 코로나19 통합뉴스룸 5시 뉴스입니다',
 '정부와 민주당이 부동산 관련 대출 규제 완화를 검토하기 시작했습니다',
 '주택 실수요자에 한해 대출 우대를 확대하는 방안이 논의되고 있습니다',
 '한편 국민의 힘은 다음 달 한미 정상회담에서 코로나19 백신을 최대한 확보할 것을 정부에 주문했습니다',
 '정부와 더불어민주당은 오늘 국회에서 비공개 당정협의를 열고 주택담보인정비율 ltv의 비율을 높여주는 우대 대상을 확대하는 방안을 논의했습니다',
 '현재 9억 원 이하 주택은 서울 같은 투기지역 등 규제 지역에선 집값의 40%만 조정대상지역은 50%를 대출받을 수 있습니다',
 '무주택자는 우대를 받아 여기에 10%를 더 빌릴 수 있는데 이런 우대를 더 많은 대상에게 확대 적용하겠다는 겁니다',
 '당정은 소득을 따져서 부채 규모를 제한하는 dsr 규제도 완화하는 방안을 앞으로 논의할 계획입니다',
 '국회 정무위 민주당 간사인 김병욱 의원은 1가구 1주택 종합부동산세 적용 기준을 현행 9억 원에서 12억 원으로 상향하는 법안을 발의했습니다',
 '민주당에서는 재보선 패배 이후 부동산 세제를 완화하자는 주장이 개별적인 차원이지만 이어지고 있습니다',
 '국민의 힘은 늦어지는 백신 수급에 문제를 제기하며 다음 달 한미정상회담에서 문재인 대통령이 코로나19 백신을 충분히 확보해야 한다고 지적했습니다',
 '국민의 힘 주호영 당 대표 권한대행은 미국 방문의 가장 중요한 의제는 백신 확보가 돼야 한다면서 좋은 코로나19 백신을 어떻게 많이 확보하느냐에 우리 외교력의 성적표가 달려있다고 강조했습니다',
 '정부가 올해 2분기에 도입하겠다고 밝혔던 모더나 백신은 상반기 확보가 어려울 전망입니다',
 '홍남기 국무총리 대행은 오늘 국회 대정부질문에서 문 대통령이 모두나 최고경영자와 화상통화를 확보했다는 모두나 백신은 어디 있느냐는 국민의 힘 김은혜 의원의 질문에 이같이 답했습니다',
 '홍 총리 대행은 모두 나눈 4천만 도스 2천만 명 분을 계약했다며

In [35]:
for i, sent in enumerate(script_list):
    if (i >= ws) & (i <= len(script_list) - ws):
        j = (i-ws)
        print(f"{score_list[j]:.2f}", '\n', sent)
    else:
        print(sent)

여러분 안녕하십니까 코로나19 통합뉴스룸 5시 뉴스입니다
정부와 민주당이 부동산 관련 대출 규제 완화를 검토하기 시작했습니다
주택 실수요자에 한해 대출 우대를 확대하는 방안이 논의되고 있습니다
-6.11 
 한편 국민의 힘은 다음 달 한미 정상회담에서 코로나19 백신을 최대한 확보할 것을 정부에 주문했습니다
-2.87 
 정부와 더불어민주당은 오늘 국회에서 비공개 당정협의를 열고 주택담보인정비율 ltv의 비율을 높여주는 우대 대상을 확대하는 방안을 논의했습니다
-5.43 
 현재 9억 원 이하 주택은 서울 같은 투기지역 등 규제 지역에선 집값의 40%만 조정대상지역은 50%를 대출받을 수 있습니다
-5.39 
 무주택자는 우대를 받아 여기에 10%를 더 빌릴 수 있는데 이런 우대를 더 많은 대상에게 확대 적용하겠다는 겁니다
-4.99 
 당정은 소득을 따져서 부채 규모를 제한하는 dsr 규제도 완화하는 방안을 앞으로 논의할 계획입니다
2.74 
 국회 정무위 민주당 간사인 김병욱 의원은 1가구 1주택 종합부동산세 적용 기준을 현행 9억 원에서 12억 원으로 상향하는 법안을 발의했습니다
-2.51 
 민주당에서는 재보선 패배 이후 부동산 세제를 완화하자는 주장이 개별적인 차원이지만 이어지고 있습니다
6.86 
 국민의 힘은 늦어지는 백신 수급에 문제를 제기하며 다음 달 한미정상회담에서 문재인 대통령이 코로나19 백신을 충분히 확보해야 한다고 지적했습니다
-4.63 
 국민의 힘 주호영 당 대표 권한대행은 미국 방문의 가장 중요한 의제는 백신 확보가 돼야 한다면서 좋은 코로나19 백신을 어떻게 많이 확보하느냐에 우리 외교력의 성적표가 달려있다고 강조했습니다
-5.13 
 정부가 올해 2분기에 도입하겠다고 밝혔던 모더나 백신은 상반기 확보가 어려울 전망입니다
-2.35 
 홍남기 국무총리 대행은 오늘 국회 대정부질문에서 문 대통령이 모두나 최고경영자와 화상통화를 확보했다는 모두나 백신은 어디 있느냐는 국민의 힘 김은혜 의원의 질문에 이같이 답했습니다
-2.61 
 

In [43]:
score_list

[-8.236818313598633,
 -8.497635841369629,
 -7.729971885681152,
 -7.541037559509277,
 -0.017026126384735107,
 -6.975645065307617,
 0.946483314037323,
 1.0367164611816406,
 -3.7296366691589355,
 -7.888616561889648,
 -0.2195427417755127,
 3.6759793758392334,
 -5.857693672180176,
 -7.514585494995117,
 -5.989588737487793,
 -3.4307117462158203,
 4.793019771575928,
 5.50775671005249,
 4.240488529205322,
 7.416803359985352,
 4.55340051651001,
 -8.771045684814453,
 -6.874622344970703]

In [36]:
test_emb = embedder.get_embeddings(test_article).transpose(1, 0).cuda()

In [37]:
test_emb.size()

torch.Size([768, 8])